In [599]:
from games_setup import *
from SBMLLint.games.som import SOM

In [597]:
simple = load_file_from_curated_data(13)
for reaction in simple.reactions:
  print(reaction)

E1: RuBP_ch + x_CO2 -> 2.00 PGA_ch; Light_on * Rbco_vm * RuBP_ch * chloroplast / (RuBP_ch + Rbco_km * (1 + PGA_ch / Rbco_KiPGA + FBP_ch / Rbco_KiFBP + SBP_ch / Rbco_KiSBP + Pi_ch / Rbco_KiPi + x_NADPH_ch / Rbco_KiNADPH))
E2: PGA_ch + ATP_ch -> BPGA_ch + ADP_ch; Light_on * PGK_v * chloroplast * (PGA_ch * ATP_ch - BPGA_ch * ADP_ch / q2)
E3: x_NADPH_ch + BPGA_ch + x_Proton_ch -> x_NADP_ch + GAP_ch + Pi_ch; Light_on * G3Pdh_v * chloroplast * (BPGA_ch * x_NADPH_ch * x_Proton_ch - x_NADP_ch * GAP_ch * Pi_ch / q3)
E4: GAP_ch -> DHAP_ch; chloroplast * TPI_v * (GAP_ch - DHAP_ch / q4)
E5: GAP_ch + DHAP_ch -> FBP_ch; F_Aldo_v * chloroplast * (DHAP_ch * GAP_ch - FBP_ch / q5)
E6: FBP_ch -> F6P_ch + Pi_ch; Light_on * FBPase_ch_vm * FBP_ch * chloroplast / (FBP_ch + FBPase_ch_km * (1 + F6P_ch / FBPase_ch_KiF6P + Pi_ch / FBPase_ch_KiPi))
E7: GAP_ch + F6P_ch -> X5P_ch + E4P_ch; chloroplast * F_TKL_v * (F6P_ch * GAP_ch - E4P_ch * X5P_ch / q7)
E8: DHAP_ch + E4P_ch -> SBP_ch; chloroplast * E_Aldo_v * (E4P_

In [598]:
for reaction in simple.reactions:
    print(reaction.category)

reaction_n_1
reaction_n_n
reaction_n_n
reaction_1_1
reaction_n_1
reaction_1_n
reaction_n_n
reaction_n_1
reaction_1_n
reaction_n_n
reaction_1_1
reaction_1_1
reaction_n_n
reaction_1_1
reaction_1_1
reaction_n_1
reaction_n_n
reaction_n_1
reaction_n_n
reaction_n_n
reaction_n_n


In [403]:
# soms = []
# for mole in simple.molecules:
#     soms.append(SOM({mole}))
# #c = SOM({simple.molecules[0]})
# #m = SOM({simple.molecules[1]})
# print(soms)

In [404]:
# class Arc(object):
    
#     def __init__(self, source, destination, reaction):
#         """
#         :param SOM source:
#         :param SOM destination:
#         :param Reaction reaction:
#         """
#         self.source = source
#         self.destination = destination
#         self.reaction = reaction

In [405]:
""" The following can be used to add multiple entries to an edge

G = nx.DiGraph()
G.add_edges_from([('a', 'b'), ('b', 'c')])
#set_edge_attributes(G, name, values)
G.add_edge('d', 'e', reaction = [1,2])
print("first edge")
print(G.edges(data=True))
a = G.get_edge_data('d', 'e')['reaction']
G.add_edge('d', 'e', reaction = a + [3, 4])
print("second")
print(G.edges(data=True))
"""

' The following can be used to add multiple entries to an edge\n\nG = nx.DiGraph()\nG.add_edges_from([(\'a\', \'b\'), (\'b\', \'c\')])\n#set_edge_attributes(G, name, values)\nG.add_edge(\'d\', \'e\', reaction = [1,2])\nprint("first edge")\nprint(G.edges(data=True))\na = G.get_edge_data(\'d\', \'e\')[\'reaction\']\nG.add_edge(\'d\', \'e\', reaction = a + [3, 4])\nprint("second")\nprint(G.edges(data=True))\n'

'\n\n  @classmethod\n  def reduce(cls, reaction):\n#     Reduces reaction using existing cls.soms\n#     :param Reaction reaction:\n#     :return reaction/False:     \n    # flag that will show whether the reaction was reduced\n    reduced = False\n    if reaction.category != cn.REACTION_n_n:\n        return reduced\n    # Reduces the reaction by examining all SOMs\n    for som in cls.soms:\n      reactants_in = deque([mole_stoich for mole_stoich in  \n                           reaction.reactants if \n                           mole_stoich.molecule in som.molecules])\n      reactants_out = [mole_stoich for mole_stoich in  \n                      reaction.reactants if \n                      mole_stoich.molecule not in som.molecules]\n      products_in = deque([mole_stoich for mole_stoich in  \n                          reaction.products if \n                          mole_stoich.molecule in som.molecules])\n      products_out = [mole_stoich for mole_stoich in  \n                     r

In [406]:
type(simple) == SimpleSBML

True

In [586]:
#ARROW = "->"
REACTION = "reaction"
LESSTHAN = "<"

class MESGraph(nx.DiGraph):
  """
  The MESGraph class represents a collection of SOMs as nodes
  and their inequality relationships as edges (arcs). 
  Mass inequality between SOMs from reactions can help us
  detect their relationship.
  Type I Error occurs when we find inequality between two molecules
  in the same SOM, because each element in a SOM has the same weight.
  Type II Error implies there is cyclism between molecules, such as
  A < B < C < ... < A, which is physically impossible. 
  """

  def __init__(self, simple=None):
    """
    :param list-SOM soms:
    """
    super(MESGraph, self).__init__()
    self.soms = self.initializeSOMs(simple)
    self.add_nodes_from(self.soms)
    self.identifier = self.makeId()
    self.type_one_error = False
    self.type_two_error = False

  def __repr__(self):
    return self.identifier

  def initializeSOMs(self, simple):
    """
    Create a list of one-molecule SOMs
    :param SimpleSBML simple:
    :return list-SOM:
    """
    soms = []
    if type(simple) == SimpleSBML:
      for molecule in simple.molecules:
        if molecule.name == cn.EMPTYSET:
          continue
        else:
          soms.append(SOM({molecule}))
    return soms
    
  def makeId(self):
    identifier = ""
    if self.edges:
      for edge in self.edges:
        identifier = identifier + str(edge[0]) + cn.ARC_ARROW + str(edge[1]) + "\n"

    for key, node in enumerate(nx.isolates(self)):
      identifier = identifier + str(node)
      if key < (len(list(nx.isolates(self)))-1):
        identifier = identifier + cn.KINETICS_SEPARATOR

    return identifier

  def getNode(self, molecule):
    """
    Search nodes in MESGraph and 
    returns SOM that includes the given molecule
    If no such SOM exists, return False
    :param Molecule molecule:
    :return SOM/False:
    """
    for som in list(self.nodes):
      for mole in som.molecules:
        if mole.name == molecule.name:
          return som
    return False

  def processUniUniReaction(self, reaction):
    """
    Process a 1-1 reaction to merge nodes.
    If no need to merge, return None.
    :param Reaction reactions:
    """
    if reaction.category != cn.REACTION_1_1:
      pass
    else:
      reactant_som = self.getNode(reaction.reactants[0].molecule)
      product_som = self.getNode(reaction.products[0].molecule)
      if reactant_som == product_som:
        return None
      else:
        new_som = reactant_som.merge(product_som)
        new_som.reactions.add(reaction)
        # TODO: if there are edges, need to also check them
        self.remove_node(reactant_som)
        self.remove_node(product_som)
        self.add_node(new_som)
        self.identifier = self.makeId()
        return new_som
    
  def processUniMultiReaction(self, reaction):
    """
    Process a 1-n reaction to add arcs.
    Since the mass of reactant is greater than
    that of each product, it adds arcs by
    addArc(source=products, destination=reactant). 
    :param Reaction reaction:
    """
    if reaction.category != cn.REACTION_1_n:
      pass
    else:
      destination = [reaction.reactants[0].molecule]
      source = [product.molecule for product in reaction.products]
      self.addArc(source, destination, reaction)
      self.identifier = self.makeId()

  def processMultiUniReaction(self, reaction):
    """
    Process a n-1 reaction to add arcs.
    Since the mass of product is greater than
    that of each reactant, it adds arcs by
    addArc(source=reactants, destination=product). 
    :param Reaction reaction:
    """
    if reaction.category != cn.REACTION_n_1:
      pass
    else:
      destination = [reaction.products[0].molecule]
      source = [reactant.molecule for reactant in reaction.reactants]
      self.addArc(source, destination, reaction)
      self.identifier = self.makeId()
    
  def addArc(self, source, destination, reaction):
    """
    Add arcs (edges) using two molecule lists (source/destination).
    :param list-Molecule source:
    :param list-Molecule destination:
    """
    arcs = itertools.product(source, destination)
    for arc in arcs:
      if not self.checkTypeOneError(arc, reaction):
        arc_source = self.getNode(arc[0])
        arc_destination = self.getNode(arc[1])
        # if there is already a preious reaction,
        if self.has_edge(arc_source, arc_destination):
          reaction_label = self.get_edge_data(arc_source, arc_destination)[REACTION]
          # if reaction.label is not already included in the attribute,
          if reaction.label not in set(reaction_label):
            reaction_label = reaction_label + [reaction.label]
        else:
          reaction_label = [reaction.label]
        self.add_edge(arc_source, arc_destination, reaction=reaction_label)
      else:
        continue
    
  def checkTypeOneError(self, arc, inequality_reaction=None):
    """
    Check Type I Error of an arc.
    If both source and destination are found
    in the same SOM, send error message and return True.
    If not, return False.
    :param tuple-Molecule arc:
    :param Reaction inequality_reaction:
    :return bool:
    """
    som1 = self.getNode(arc[0])
    som2 = self.getNode(arc[1])
    if som1 == som2:
      print("\nWe have a Type I Error...\n")
      print(arc[0], " and ", arc[1], " have the same weight by")
      for equality_reaction in list(som1.reactions):
        print(equality_reaction)
      print("\nHowever, reaction \"", inequality_reaction, 
            "\" implies ", arc[0], LESSTHAN, arc[1])
      print("We cannot add the arc: ", arc[0], cn.ARC_ARROW, arc[1])
      print()
      if not self.type_one_error:
        self.type_one_error = True
      return True
    else:
      return False

  def checkTypeTwoError(self):
    """
    Check Type II Error (cycles) of a MESGraph.
    If there is at least one cycle, 
    report an error message, related reactions
    and return True.
    If there is no cycle, return False. 
    :return bool:
    """
    graph = nx.DiGraph()
    graph.add_edges_from(self.edges)
    cycles = list(nx.simple_cycles(graph))
    if len(cycles) == 0:
      print("No cycles! No Type II Error!")
      return False
    else:
      print("We have a Type II Error...\n")
      for cycle in cycles:
        cycle_nodes = []
        for node in cycle:
          cycle_nodes.append(node)
        for node in cycle_nodes:
          print(node, LESSTHAN, end=" ")
        print(cycle_nodes[0], "\n")
        #
        for node_idx in range(0, len(cycle_nodes)-1):
          arc_source = cycle_nodes[node_idx]
          arc_destination = cycle_nodes[node_idx+1]
          print(arc_source, cn.ARC_ARROW, arc_destination, " by")
          reaction_label = self.get_edge_data(arc_source, arc_destination)[REACTION]
          for r_label in reaction_label:
            print(r_label + "\n")
        # last arc which completes the cycle
        arc_source = cycle_nodes[len(cycle_nodes)-1]
        arc_destination = cycle_nodes[0]
        print(arc_source, cn.ARC_ARROW, arc_destination, " by")
        reaction_label = self.get_edge_data(arc_source, arc_destination)[REACTION]
        for r_label in reaction_label:
          print(r_label + "\n")
      #     
      if not self.type_two_error:
        self.type_two_error = True
      return True

  def analyze(self, reactions):
    """
    Sort list of reactions and process them.
    Add arcs or sending error messages using
    checkTypeOneError or checkTypeTwoError.
    :param list-Reaction reactions:
    """
    # Associate the reaction category with the function
    # that processes that category
    reaction_dic = {
        cn.REACTION_1_1: self.processUniUniReaction,
        cn.REACTION_1_n: self.processUniMultiReaction,
        cn.REACTION_n_1: self.processMultiUniReaction,
        }
    # Process each type of reaction
    for category in reaction_dic.keys():
      for reaction in [r for r in reactions if r.category == category]:
        func = reaction_dic[category]
        func(reaction)
    
    # check checkTypeTwoError:
    self.checkTypeTwoError()
    return self

In [587]:
simple.reactions

[Reaction1: Br + BrO3 -> HBrO2 + HOBr; Br * BrO3 * k1 * BZ,
 Reaction2: Br + HBrO2 -> 2.00 HOBr; Br * HBrO2 * k2 * BZ,
 Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2; BrO3 * HBrO2 * k3 * BZ,
 Reaction4: 2.00 HBrO2 -> BrO3 + HOBr; pow(HBrO2, 2) * k4 * BZ,
 Reaction5: Ce -> Br; Ce * k5 * BZ]

In [588]:
simple = load_file_from_curated_data(40)
m = MESGraph(simple)
reactions = []
for reaction in [r for r in simple.reactions if r.category!=cn.REACTION_BOUNDARY]:
  if (reaction.reactants[0].molecule.name == cn.EMPTYSET) | \
      (reaction.products[0].molecule.name == cn.EMPTYSET):
   continue
  else:
    reactions.append(reaction)
# for reaction in reactions:
#   print(reaction)
z = m.analyze(reactions)
# for arc in m.edges:
#   print(arc[0], "->", arc[1])
# if z.type_two_error:
#   print("Model " + str(40) + " has a type two error")

We have a Type II Error...

{HBrO2} < {HOBr} < {HBrO2} 

{HBrO2} -> {HOBr}  by


AttributeError: 'MESGraph' object has no attribute 'simple'

In [560]:
m.nodes

NodeView(({BrO3}, {HBrO2}, {HOBr}, {Br, Ce}))

In [552]:
dummy_reaction = simple.reactions[0]
m.addArc([simple.getMolecule('MKK')], [simple.getMolecule('MKKK')], dummy_reaction)
m.addArc([simple.getMolecule('MKKK')], [simple.getMolecule('MAPK')], dummy_reaction)
m.addArc([simple.getMolecule('MAPK')], [simple.getMolecule('MKK')], dummy_reaction)
m.edges

OutEdgeView([({MKKK, MKKK_P}, {MAPK, MAPK_P, MAPK_PP}), ({MKK, MKK_P, MKK_PP}, {MKKK, MKKK_P}), ({MAPK, MAPK_P, MAPK_PP}, {MKK, MKK_P, MKK_PP})])

In [505]:
edge1 = list(m.edges)[1]
edge2 = list(m.edges)[2]
print(edge1)
print(edge2)

({HBrO2}, {HOBr})
({HOBr}, {HBrO2})


In [554]:
G = nx.DiGraph()
G.add_edges_from(m.edges)
list(nx.simple_cycles(G))

[[{MKK, MKK_P, MKK_PP}, {MKKK, MKKK_P}, {MAPK, MAPK_P, MAPK_PP}]]

In [556]:
for cycle in list(nx.simple_cycles(G)):
  for node in cycle:
    print(node)


{MKK, MKK_P, MKK_PP}
{MKKK, MKKK_P}
{MAPK, MAPK_P, MAPK_PP}


In [387]:
m.nodes

NodeView(({Pc1, Pc2, Po1, Po2},))

In [232]:
for react in simple.reactions:
    print(react, "\n")

SHMT: FH4 + serine -> CH2FH4; cell * (Vm * (serine / Km2 / (1 + serine / Km2)) * (FH4 / Km1) / (1 + FH4 / Km1)) 

SHMTr: CH2FH4 -> FH4; cell * (Vm * (glycine / Km2 / (1 + glycine / Km2)) * (CH2FH4 / Km1) / (1 + CH2FH4 / Km1)) 

HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4; cell * hp * FH4 * HCHO 

CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO; cell * hl * CH2FH4 

MTHFR: CH2FH4 + NADPH -> CH3FH4; cell * (Vm * CH2FH4 * NADPH / (NADPH * CH2FH4 + CH2FH4 * Km2 + (NADPH + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 + FH2f / Ki1))) 

MTR: CH3FH4 + homocysteine -> FH4; cell * (Vm * (homocysteine / Km2 / (1 + homocysteine / Km2)) * (CH3FH4 / Km1) / (1 + CH3FH4 / Km1)) 

HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4; cell * (Vm / ((1 + Km1 / FH4) * (1 + Km2 / ATP) * (1 + Km3 / formate))) 

GARFT: CHOFH4 + GAR -> FGAR + FH4; cell * (Vm * CHOFH4 * GAR / (GAR * CHOFH4 + CHOFH4 * Km2 + (GAR + Km2) * Km1 * (1 + MTX1 / Ki21 + MTX2 / Ki22 + MTX3 / Ki23 + MTX4 / Ki24 + MTX5 / Ki25 +

In [233]:
m.edges

OutEdgeView([])

In [234]:
unimulti_reaction = simple.reactions[3]
destination = [reactant.molecule for reactant in unimulti_reaction.reactants]
source = [produce.molecule for produce in unimulti_reaction.products]
unimulti_reaction

CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO; cell * hl * CH2FH4

In [235]:
m.edges(data=True)

OutEdgeDataView([])

In [236]:
m.addArc(source, destination, unimulti_reaction)

In [237]:
m.edges(data=True)

OutEdgeDataView([({FH4}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']}), ({HCHO}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']})])

In [212]:
fh4 = simple.molecules[0]
ch2fh4 = simple.molecules[2]
print(fh4)
print(ch2fh4)

FH4
CH2FH4


In [213]:
m.has_edge(m.getNode(fh4), m.getNode(ch2fh4))

True

In [214]:
m.get_edge_data(m.getNode(fh4), m.getNode(ch2fh4))['reaction']

['CH2FH4toHCHO']

In [215]:
m.processUniMultiReaction(unimulti_reaction)
m.edges

OutEdgeView([({FH4}, {CH2FH4}), ({HCHO}, {CH2FH4})])

In [217]:
m.edges(data=True)

OutEdgeDataView([({FH4}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']}), ({HCHO}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']})])

In [218]:
multiuni_reaction = simple.reactions[0]
multiuni_reaction

SHMT: FH4 + serine -> CH2FH4; cell * (Vm * (serine / Km2 / (1 + serine / Km2)) * (FH4 / Km1) / (1 + FH4 / Km1))

In [219]:
m.processMultiUniReaction(multiuni_reaction)
m.edges

OutEdgeView([({FH4}, {CH2FH4}), ({serine}, {CH2FH4}), ({HCHO}, {CH2FH4})])

In [220]:
m.edges(data=True)

OutEdgeDataView([({FH4}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO', 'SHMT']}), ({serine}, {CH2FH4}, {'reaction': ['SHMT']}), ({HCHO}, {CH2FH4}, {'reaction': ['CH2FH4toHCHO']})])

In [221]:
m.processUniUniReaction(reaction)

In [222]:
m.nodes

NodeView(({FH4}, {serine}, {CH2FH4}, {HCHO}, {NADPH}, {CH3FH4}, {homocysteine}, {formate}, {ATP}, {CHOFH4}, {GAR}, {FGAR}, {AICAR}, {NADP}, {dUMP}, {FH2f}, {FFH2}, {MTX1}, {MTX2}, {MTX3}, {MTX4}, {MTX5}, {EMTX}, {DHFRf}, {MTX1b}, {MTX2b}, {MTX3b}, {MTX4b}, {MTX5b}))

In [18]:
m.analyze(Reaction.reactions)
m.nodes

NodeView(({NAD}, {halfglucose}, {NADH}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {Butanediol}, {O2}, {AcLac, AcetoinIn, AcetoinOut, pyruvate}, {ADP, ATP}))

In [19]:
m.edges

OutEdgeView([({NADH}, {NAD}), ({O2}, {NAD})])

In [20]:
m

{NADH}->{NAD}
{O2}->{NAD}
{halfglucose};{lactate};{CoA};{AcCoA};{PO4};{AcP};{Ac};{AcO};{EtOH};{Butanediol};{AcLac, AcetoinIn, AcetoinOut, pyruvate};{ADP, ATP}

In [453]:
m.edges

OutEdgeView([({Gly}, {DFG, E1, E2}), ({Gly}, {Mel}), ({MG}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}), ({Fru}, {DFG, E1, E2}), ({AA, Cn, FA, Glu, Man, _1DG, _3DG}, {DFG, E1, E2}), ({AA, Cn, FA, Glu, Man, _1DG, _3DG}, {Mel})])

In [456]:
list(nx.isolates(m))

[]

In [424]:
identifier = ""
if m.edges:
    for edge in m.edges:
        identifier = identifier + str(edge[0]) + "->" + str(edge[1]) + "\n"
    
for key, node in enumerate(nx.isolates(m)):
    identifier = identifier + str(node)
    if key < (len(list(nx.isolates(m)))-1):
        identifier = identifier + cn.KINETICS_SEPARATOR
print(identifier)

{Gly}->{DFG, E1, E2}
{Gly}->{Mel}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}



{DFG};{E1};{E2};{Gly};{Cn};{_3DG};{FA};{_1DG};{AA};{Man};{Glu};{Mel};{MG};{Fru}

In [323]:
str(edge[0])

'{Gly}'

In [343]:
nodes = list(m.nodes)
print(nodes)

[{Gly}, {Mel}, {MG}, {Fru}, {DFG, E1, E2}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}]


In [373]:
for x in nx.isolates(m):
    print(x)
    print(type(x))

{ATP, Ade, DNA, GTP, Gua, HX, IMP, PRPP, R5P, RNA, SAM, SAMP, UA, XMP, Xa, dATP, dGTP}
<class 'SBMLLint.games.som.SOM'>


In [341]:
nx.ancestors(m, list(m.edges)[0][1])

{{Fru}, {MG}, {Gly}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}}

In [339]:
list(m.edges)[0][0]

{Gly}

In [374]:
m

;{ATP}+{HX}+{Ade}+{dATP}+{DNA}+{IMP}+{PRPP}+{RNA}+{SAMP}+{dGTP}+{Gua}+{GTP}+{XMP}+{Xa}+{SAM}+{R5P}+{UA}+

In [376]:
nx.isolates(m)

<generator object isolates.<locals>.<genexpr> at 0x1521ab2c50>

In [22]:
a = {1}.union({2})

In [23]:
a

{1, 2}

In [24]:
a.add(3)

In [25]:
a

{1, 2, 3}

In [26]:
SOM.soms

[{E2},
 {Gly},
 {Cn},
 {_3DG},
 {FA},
 {_1DG},
 {AA},
 {Man},
 {Glu},
 {Mel},
 {MG},
 {Fru}]